In [ ]:
# imports
from astropy.time import Time
from astroplan import Observer
import numpy
import pandas
from urllib import parse as urlparse, request
import json
from IPython.display import display, Markdown

import lsst.daf.butler as dafButler

In [ ]:
# for template variables
# log_date = Time({{ params.log_date }})
# url_base = {{ params.url_base }}

In [ ]:
from dateutil import parser
log_date_param = '2022-04-05'
# need to account for daylight savings
log_date = Time(parser.isoparse(f'{log_date_param}T12:00:00.00-04:00'))
url_base = f'https://lsst.ncsa.illinois.edu/~krughoff/data/auxtel_nightly/{log_date_param}/'

In [ ]:
def make_pretty(styler):
    styler.format(lambda x: f'{x:.2f}' if (isinstance(x, float) or isinstance(x, int)) else x)
    styler.hide(axis='index')
    styler.background_gradient(axis=None, vmin=0, vmax=12., cmap='YlGnBu')
    return styler

def make_pretty_times(styler):
    styler.format(lambda x: f'{x:.2f}' if (isinstance(x, float) or isinstance(x, int)) else x)
    styler.background_gradient(axis=None, vmin=0, vmax=6., cmap='YlGnBu')
    styler.format_index(lambda v: v.strftime("UTC %c"))
    return styler

In [ ]:
def get_dict(url_base, fn):
    with request.urlopen(urlparse.urljoin(url_base, fn)) as response:
        txt = response.read()
    return json.loads(txt)

In [ ]:
def get_text_blob(url_base, fn):
    with request.urlopen(urlparse.urljoin(url_base, fn)) as response:
        txt = response.read()
    return txt.decode('utf-8')

In [ ]:
def get_df(url_base, fn, parse_dates=True):
    return pandas.read_csv(urlparse.urljoin(url_base, fn), index_col=0, parse_dates=parse_dates,
                           keep_default_na=False)

In [ ]:
title = f'# {log_date_param} Aux-Tel Night Log'
display(Markdown(title))

In [ ]:
obs_dict = get_dict(url_base, 'observers.txt')
obs_list = f'''
- Observers (on-summit): {obs_dict['observers']}
- Authors(s): {obs_dict['authors']}
- Daytime Support: {obs_dict['day']}
- Online support: {obs_dict['online']}
'''
display(Markdown(obs_list))

### Almanac:

In [ ]:
gemini = Observer.at_site('gemini_south')
# we could get pressure and temp from the EFD to make refraction calculations more accurate
moon_rise = gemini.moon_rise_time(log_date, which='next')
moon_set = gemini.moon_set_time(log_date, which='next')
if moon_set < moon_rise:
    moon_rise = gemini.moon_rise_time(log_date, which='previous')
sun_set = gemini.sun_set_time(log_date, which='next')
sun_rise = gemini.sun_rise_time(log_date, which='next')
evening_twilight = gemini.twilight_evening_nautical(log_date, which='next')
morning_twilight = gemini.twilight_morning_nautical(log_date, which='next')

In [ ]:
almanac_str = f'''Moonrise: {moon_rise.strftime("UTC %c")}  
Sunset: {sun_set.strftime("UTC %c")}  
End of nautical twilight: {evening_twilight.strftime("UTC %c")}  
Moonset: {moon_set.strftime("UTC %c")}  
Beginning of morning nautical twilight: {morning_twilight.strftime("UTC %c")}  
Sunrise: {sun_rise.strftime("UTC %c")}  
'''
display(Markdown(almanac_str))

### List of current software

* Software/environment:
* Scheduler environment:

### Important Operations Status:

In [ ]:
ops_stat_str = get_text_blob(url_base, 'ops_status.txt')
display(Markdown(ops_stat_str))

### Outstanding Daytime Tasks:

In [ ]:
tasks = get_df(url_base, 'day_tasks.csv', parse_dates=False)
display(tasks.style.pipe(make_pretty))

### Plan for the night


In [ ]:
night_plan_str = get_text_blob(url_base, 'night_plan.txt')
display(Markdown(night_plan_str))

### Nighttme summary

In [ ]:
night_summary_str = get_text_blob(url_base, 'night_summary.txt')
display(Markdown(night_summary_str))

Snapshot from tonight's imaging survey **??????**

### Time accounting:

In [ ]:
ta = get_df(url_base, 'time_accounting.csv', parse_dates=False)
display(ta.style.pipe(make_pretty))

All times reported in hours.  
\* Defined as the total number of hours between 12 degrees (nautical) evening and morning twilights.  
** Defined as time spent for observers/support to better understand the system.

### Fault reports:

In [ ]:
fr = get_df(url_base, 'faults.csv')
display(fr.style.pipe(make_pretty_times))

### Weather summary:

In [ ]:
weather = get_df(url_base, 'weather.csv')
display(weather.style.pipe(make_pretty_times))

### Detailed Night Log:

In [ ]:
nl = get_df(url_base, 'night_log.csv')
display(nl.style.pipe(make_pretty_times))

### Exposure Log:
[log](https://confluence.lsstcorp.org/download/attachments/182427721/LATISS_20220405_Log%20%281%29.csv?version=1&modificationDate=1649261980000&api=v2)

> unfortunately, the above is not parameterizeable because the attachment id isn't something I know how to discover

### Below is fake because we are working on data-int, so don't have access to the nightly LATISS logs.  We are using DP0.2 instead.

In [ ]:
repo = 'dp02'
butler = dafButler.Butler(repo, collections='2.2i/runs/DP0.2')
registry = butler.registry
recs = registry.queryDimensionRecords(bind={'day': 20211231}, where="exposure.day_obs = day", element='exposure', instrument='LSSTCam-imSim')
data = {}
data['Exposure start'] = []
data['Physical filter'] = []
data['Exposure time'] = []
for rec in recs:
    if rec.observation_type == 'science':
        data['Exposure start'].append(rec.timespan.begin)
        data['Physical filter'].append(rec.physical_filter)
        data['Exposure time'].append(rec.exposure_time)
        
exp_log = pandas.DataFrame.from_dict(data)
exp_log = exp_log.set_index('Exposure start')
display(exp_log.style.pipe(make_pretty_times))